Import necessary libraries

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 199 kB 54.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=910b9df8ca64f6aedc775c9ea46c61f432659bc2f5e37c14e02f14d917bf5a5a
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
import numpy as np 
import pandas as pd
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from google.colab import drive
from pyspark.ml.feature import StringIndexer

In [4]:
sc = SparkContext()
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

In [49]:
drive.mount('/drive')
ratings = spark.read.csv("/drive/MyDrive/585_project/data_200,000.csv",sep = ',', header = True)
ratings.show()

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
+------+--------------------+-------+
|userID|             movieID|ratings|
+------+--------------------+-------+
|340626|acapulco_+prima+s...|      4|
| 26447|         otello+1986|      3|
|172600|          holes+2003|      2|
|314533|my+brother+the+te...|      4|
|249289|sommer+der+gaukle...|      2|
|186589|  aces+n+eights+2008|      3|
|278499|children+on+their...|      4|
| 95207|       excision+2012|      3|
| 48275|       outsider+1997|      4|
| 69998|rosas+hllenfahrt+...|      4|
|102246|los+signos+del+zo...|      4|
|427110|my+neighbor+totor...|      4|
|379843|        vincent+1982|      5|
|358920|     mirrormask+2005|      3|
|339159|this+is+spinal+ta...|      4|
| 91269|forbidden+planet+...|      3|
|447391|           gigi+1958|      2|
|410073|whats+up_+scarlet...|      3|
| 98060|the+heart+of+the+...|      5|
|335447|      44500+max+2009|      4|
+------+-----

In [6]:
ratings.printSchema()

root
 |-- userID: string (nullable = true)
 |-- movieID: string (nullable = true)
 |-- ratings: string (nullable = true)



In [50]:
ratings = ratings. \
    withColumn('userId', col('userID').cast('integer')).\
    withColumn('movieId', col('movieID').cast('string')).\
    withColumn('rating', col('ratings').cast('float')).\
    drop('ratings')
    #drop('_c3').\
    #drop('_c0').\
    #drop('_c1').\
    #drop('_c2')
ratings.show()

+------+--------------------+------+
|userId|             movieId|rating|
+------+--------------------+------+
|340626|acapulco_+prima+s...|   4.0|
| 26447|         otello+1986|   3.0|
|172600|          holes+2003|   2.0|
|314533|my+brother+the+te...|   4.0|
|249289|sommer+der+gaukle...|   2.0|
|186589|  aces+n+eights+2008|   3.0|
|278499|children+on+their...|   4.0|
| 95207|       excision+2012|   3.0|
| 48275|       outsider+1997|   4.0|
| 69998|rosas+hllenfahrt+...|   4.0|
|102246|los+signos+del+zo...|   4.0|
|427110|my+neighbor+totor...|   4.0|
|379843|        vincent+1982|   5.0|
|358920|     mirrormask+2005|   3.0|
|339159|this+is+spinal+ta...|   4.0|
| 91269|forbidden+planet+...|   3.0|
|447391|           gigi+1958|   2.0|
|410073|whats+up_+scarlet...|   3.0|
| 98060|the+heart+of+the+...|   5.0|
|335447|      44500+max+2009|   4.0|
+------+--------------------+------+
only showing top 20 rows



In [52]:
from pyspark.sql.functions import dense_rank
from pyspark.sql.window import Window

ratings = ratings.withColumn("movieIndex", dense_rank().over(Window.orderBy("movieId")))
ratings.show()

+------+--------------------+------+----------+
|userId|             movieId|rating|movieIndex|
+------+--------------------+------+----------+
|406568|          ++++++1959|   4.0|         1|
|362230|          ++++++1959|   4.0|         1|
| 17433|          ++++++1959|   4.0|         1|
|435477|          ++++++1959|   4.0|         1|
|190145|          ++++++1959|   4.0|         1|
|158290|          ++++++1959|   3.0|         1|
| 44682|          ++++++1959|   4.0|         1|
|369213|            ++++2013|   3.0|         2|
| 66335|       +la+mode+1993|   3.0|         3|
|210367|       +la+mode+1993|   3.0|         3|
| 79536|       +la+mode+1993|   3.0|         3|
|363147|     +laventure+2008|   3.0|         4|
| 30664|     +laventure+2008|   4.0|         4|
|123697|     +laventure+2008|   3.0|         4|
|350111|    +nos+amours+1983|   4.0|         5|
|389308|    +nos+amours+1983|   4.0|         5|
|230428|+nous+la+libert+1931|   5.0|         6|
|385620|+nous+la+libert+1931|   3.0|    

In [37]:
from pyspark.sql.functions import split

In [53]:
table = ratings.select('movieID','movieIndex')
table.show()

+--------------------+----------+
|             movieID|movieIndex|
+--------------------+----------+
|          ++++++1959|         1|
|          ++++++1959|         1|
|          ++++++1959|         1|
|          ++++++1959|         1|
|          ++++++1959|         1|
|          ++++++1959|         1|
|          ++++++1959|         1|
|            ++++2013|         2|
|       +la+mode+1993|         3|
|       +la+mode+1993|         3|
|       +la+mode+1993|         3|
|     +laventure+2008|         4|
|     +laventure+2008|         4|
|     +laventure+2008|         4|
|    +nos+amours+1983|         5|
|    +nos+amours+1983|         5|
|+nous+la+libert+1931|         6|
|+nous+la+libert+1931|         6|
|+nous+la+libert+1931|         6|
|+nous+la+libert+1931|         6|
+--------------------+----------+
only showing top 20 rows



In [58]:
final_table = table.select('movieID','movieIndex').distinct()
final_table.show()

+--------------------+----------+
|             movieID|movieIndex|
+--------------------+----------+
|          ++++++1959|         1|
|            ++++2013|         2|
|       +la+mode+1993|         3|
|     +laventure+2008|         4|
|    +nos+amours+1983|         5|
|+nous+la+libert+1931|         6|
|+propos+de+nice+1930|         7|
|...and+god+create...|         8|
|...and+justice+fo...|         9|
|...and+the+pursui...|        10|
|...tick...+tick.....|        11|
|            .45+2006|        12|
|  009+re+cyborg+2012|        13|
|           0605+2004|        14|
|1+-+nenokkadine+2014|        15|
|              1+2013|        16|
|1+knights+-+in+se...|        17|
|  10+000+timmar+2014|        18|
|             10+1979|        19|
|10+items+or+less+...|        20|
+--------------------+----------+
only showing top 20 rows



In [59]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- movieIndex: integer (nullable = false)



In [64]:
#example of getting from the table.
final_table.where(final_table.movieIndex == 1).show()

+----------+----------+
|   movieID|movieIndex|
+----------+----------+
|++++++1959|         1|
+----------+----------+



In [67]:
final_table.write.option("header",True).option("delimiter",",").csv("/drive/MyDrive/585_project/lookuptable")

In [68]:
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 3333) 
# ratings here is the the Spark dataframe type

In [69]:
als = ALS(userCol="userId", itemCol="movieIndex" \
          ,ratingCol="rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

Tuning the model

In [70]:
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 40, 70, 100, 130, 160, 200]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)
print(cv)

CrossValidator_5ae4f75bca13


In [ ]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train) 
#train can be changed to ratings if we do not want to test it and want to use all data for validation

#Extract best model from the cv model above
best_model = model.bestModel

In [ ]:
# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

In [ ]:
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

In [ ]:
test_predictions.show()

In [ ]:
best_model.save("/drive/MyDrive/585_project/ALS")

Make Recommendations

In [ ]:
nrecommendations = best_model.recommendForAllUsers(20) #top 20 recommandations
nrecommendations.limit(20).show()
#nrecommendations4user_subset = best_model.recommendForUserSubset(user_subset, 3)
# user subset will be another data frame object. 

Save the trained model in pkl file

In [ ]:
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)